In [1]:
#### Loading the packages to be used
import pandas as pd
import numpy as np
from __future__ import print_function  # Python 2 and 3
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model
ols = linear_model.LinearRegression()

In [121]:
drug= pd.read_csv('drug.csv',dtype={'NDC': str})
priceInfo=drug.copy()

In [34]:
drug['PACKAGEDESCRIPTION'].unique()

array([ '1 VIAL, SINGLE-DOSE in 1 PACKAGE (0009-3073-01)  > 1 mL in 1 VIAL, SINGLE-DOSE',
       '1 BOTTLE in 1 CARTON (0009-3116-01)  > 30 mL in 1 BOTTLE',
       '10 POUCH in 1 BOX (10019-553-01)  > 1 PATCH in 1 POUCH > 3 d in 1 PATCH',
       ...,
       '30 TABLET, FILM COATED, EXTENDED RELEASE in 1 BOTTLE (99207-466-30)',
       '30 TABLET, FILM COATED, EXTENDED RELEASE in 1 BOTTLE (99207-467-30)',
       '1 TUBE in 1 CARTON (99207-850-60)  > 60 g in 1 TUBE'], dtype=object)

In [12]:
drug.dtypes

PRODUCTID_x                                   object
PRODUCTNDC                                    object
NDCPACKAGECODE                                object
PACKAGEDESCRIPTION                            object
PRODUCTID_y                                   object
PRODUCTTYPENAME                               object
PROPRIETARYNAME                               object
PROPRIETARYNAMESUFFIX                         object
NONPROPRIETARYNAME                            object
DOSAGEFORMNAME                                object
ROUTENAME                                     object
STARTMARKETINGDATE                           float64
ENDMARKETINGDATE                             float64
MARKETINGCATEGORYNAME                         object
APPLICATIONNUMBER                             object
LABELERNAME                                   object
SUBSTANCENAME                                 object
ACTIVE_NUMERATOR_STRENGTH                     object
ACTIVE_INGRED_UNIT                            

In [122]:
priceInfo.shape

(13102, 35)

In [123]:
priceInfo = priceInfo.loc[:,['DOSAGEFORMNAME'] + ['ROUTENAME'] + ['STARTMARKETINGDATE'] + ['MARKETINGCATEGORYNAME']+ ['LABELERNAME']  + ['PHARM_CLASSES'] + ['DEASCHEDULE'] +['OTC']+['NADAC_Per_Unit']+['Classification_for_Rate_Setting']]
def Dummify(column):
    column_values = pd.get_dummies(priceInfo[column], prefix=column, prefix_sep='__')
    return pd.concat([priceInfo.drop(column,axis=1),column_values],axis=1)

priceInfo = Dummify('DOSAGEFORMNAME')
priceInfo = Dummify('ROUTENAME')
priceInfo = Dummify('LABELERNAME')
priceInfo = Dummify('PHARM_CLASSES')
priceInfo = Dummify('DEASCHEDULE')
priceInfo = Dummify('Classification_for_Rate_Setting')
priceInfo = Dummify('OTC')
priceInfo = Dummify('MARKETINGCATEGORYNAME')


In [124]:
priceInfo.shape

(13102, 1145)

In [125]:
y = priceInfo["NADAC_Per_Unit"]
x = priceInfo.ix[:, priceInfo.columns != 'NADAC_Per_Unit']

In [17]:
# use cross validation
# from sklearn.model_selection import KFold, cross_val_score
# kfold = KFold(n_splits=3)
# [ols.fit(x[train], y[train]).score(x[test], y[test]) for train, test in k_fold.split(x)]

NameError: name 'k_fold' is not defined

In [35]:
# cross validation score - NOT work
import sklearn.model_selection as ms
scores = ms.cross_val_score(estimator=ols, X=x, y=y, cv=5)
scores

array([ -5.44022204e+01,  -2.49161686e-01,  -5.28834289e+09,
        -1.70917816e+13,  -1.04435699e+12])

In [133]:
# BEST SPLIT SO FAR - use 0.1 train and test split
try:  # train_test_split was moved in 0.18.0
    from sklearn.model_selection import train_test_split
except:  # Following import works through 0.19 but outputs a warning in 0.18
    from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [13]:
X_test
y_test

4347      0.02258
7733      3.24485
3545      0.11749
1237      0.84565
720       0.02794
8523      0.01173
9314      0.10262
9910      7.50384
1739      0.02521
2452      0.24712
12005     0.47341
1178      0.16945
11791     0.58089
10075     0.28330
379       0.18324
1935      0.29530
7039      3.03085
6866      0.17115
4333      0.01426
12875     0.05777
4339      0.04692
10800     0.20276
11670     0.56360
6497     29.70500
2150      0.07607
902       0.29317
4257     13.12417
1236      0.84565
12607     0.10969
6023      0.24949
           ...   
7377      0.09929
8360      0.29975
8204      0.08403
11834     0.04703
6939      0.47768
2631      0.25059
8731      0.15465
3910      5.22662
4750     11.57547
732       5.64150
245       0.11266
2057      0.96209
13059     0.54243
10090     0.78737
12662     1.16384
6604      2.81522
7539     13.37750
8644      0.56360
6286      0.09909
12153     1.05668
673       0.01319
2117      0.03887
3128      0.56973
5151      0.07494
476       

In [134]:
ols.fit(X_train, y_train)
print("R^2 for train set: %f" %ols.score(X_train, y_train))

print('-'*50)

print("R^2 for test  set: %f" %ols.score(X_test, y_test))

# BEST SCORE SO FAR
# R^2 for train set: 0.919335
# --------------------------------------------------
# R^2 for test  set: 0.994431

R^2 for train set: 0.919335
--------------------------------------------------
R^2 for test  set: 0.994431


In [136]:
from sklearn.metrics import mean_squared_error

#rms = np.sqrt(mean_squared_error(y_actual, y_predicted))
print("Mean squared root error: %.2f"
      % np.sqrt(((ols.predict(X_test) - y_test) ** 2).mean()))
        
# Mean squared root error: 37.79

Mean squared root error: 37.79


In [93]:
# ====================================================================================
# ====================================================================================
# NOT WORK- try to add more features 

# keyfeatures=(['DOSAGEFORMNAME','ROUTENAME','STARTMARKETINGDATE','MARKETINGCATEGORYNAME',
#             'PHARM_CLASSES','DEASCHEDULE','OTC','Explanation_Code','NADAC_Per_Unit'])

keyfeatures=(['DOSAGEFORMNAME','ROUTENAME','STARTMARKETINGDATE','LABELERNAME',
            'PHARM_CLASSES','DEASCHEDULE','NADAC_Per_Unit'])

drugPrice = drug.loc[:,keyfeatures]


In [84]:
#drug.MARKETINGCATEGORYNAME

0                          NDA
1                          NDA
2                          NDA
3                          NDA
4                          NDA
5                          NDA
6                          NDA
7                          NDA
8                          NDA
9                          NDA
10                         NDA
11                         NDA
12                         NDA
13                         NDA
14                         NDA
15                         NDA
16                         NDA
17                         NDA
18                         NDA
19                        ANDA
20                         NDA
21                         NDA
22                         NDA
23                        ANDA
24                        ANDA
25                        ANDA
26                        ANDA
27                        ANDA
28                        ANDA
29                        ANDA
                 ...          
13072                     ANDA
13073   

In [94]:
#drugPrice = Dummify(dummiedV)
drugT=pd.concat([drugPrice, pd.get_dummies(drugPrice.loc[:,drugPrice.dtypes=="object"])], axis=1)

In [95]:
drop_old=keyfeatures[:-1]
drop_old

['DOSAGEFORMNAME',
 'ROUTENAME',
 'STARTMARKETINGDATE',
 'LABELERNAME',
 'PHARM_CLASSES',
 'DEASCHEDULE']

In [96]:
drugT=drugT.drop(drop_old,axis=1)

In [97]:
drugT.shape

(13102, 1131)

In [98]:
y = drugT["NADAC_Per_Unit"]
x = drugT.ix[:, drugT.columns != 'NADAC_Per_Unit']

In [99]:
try:  # train_test_split was moved in 0.18.0
    from sklearn.model_selection import train_test_split
except:  # Following import works through 0.19 but outputs a warning in 0.18
    from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [100]:
ols.fit(X_train, y_train)
print("R^2 for train set: %f" %ols.score(X_train, y_train))

print('-'*50)

print("R^2 for test  set: %f" %ols.score(X_test, y_test))

R^2 for train set: 0.917570
--------------------------------------------------
R^2 for test  set: -142324822316929646592.000000
